In [3]:
# KBS 뉴스 Crawling

from bs4 import BeautifulSoup as bs
# from selenium import webdriver as wd
# from selenium.webdriver.chrome.service impaort Service
# from webdriver_manager.chrome import ChromeDriverManager
import requests
# import subprocess
import numpy as np
import pandas as pd

from time import sleep

In [1]:
url = "https://news.kbs.co.kr"
url_main = "https://news.kbs.co.kr/news/list.do?ref=pSiteMap#20220921&1"
url_target = "https://news.kbs.co.kr/news/list.do?ref=pSiteMap#20220921&1"

In [2]:
columns = ['title', 'headline', 'date', 'news_link', 'content', 'category']
data = []

In [23]:
currnet_page = requests.get(url=url_main).text
soup = bs(currnet_page, 'html.parser')
# content = soup.find('ul', {'class' : 'list-type list-thumb'})
article = soup.find_all('li', {'style' : 'thumbnailNewsList'})

print(article)
# content

None


In [4]:
print('----- Start -----')

n_page = 0

while True:
    current_page = requests.get(url=url_main) # main page(idx=1)
    while True:
        if current_page.status_code == 200:
            current_page = current_page.text
            break
        else:
            print(current_page.status_code)
        
    soup = bs(current_page, 'html.parser')
    sleep(0.5)
    
    link = soup.find('link', {'rel' : 'canonical'})['href']
    date = link.split('?pageDate=')[-1]
    if date == '20220920':
        break
    
    next_date = list(soup.find('ol',  {'class' : 'mdp_date_list'})) # ['\n', date]
    next_date = [e for e in next_date if e != '\n'][1] # remove '\n'
    next_url = next_date.find('a')['href']
    
    idx_p = 1
    
    n_page += 1
    print()
    print('page: ' + str(n_page))
    print()
    
    while True:
        print()
        print('pageIdx=' + str(idx_p))
        print()
        
        current_page = requests.get(url=url_target) # next page(idx > 1)
        while True:
            if current_page.status_code == 200:
                current_page = current_page.text
                break
            else:
                print(current_page.status_code)
            
        soup = bs(current_page, 'html.parser')
        sleep(5)
        
        flag = soup.find('div', {'class' : 'w_news_list type_issue'}).text.strip() # content presence
        if flag == '해당 일자의 뉴스 정보가 없습니다.':
            break
        
        list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'})) # current page news
        
        for news in list_news:
            try:
                title = news.find('strong').text # title
                headline = news.find('span', {'class' : 'read'}).text # headline
                # thumb = news.find('img')['src'] # thumbnail
                date = news.find('span', {'class' : 'date'}).text # date
                link_news = news.find('meta')['itemid'] # news link
            except AttributeError as e:
                title = None
                headline = None
                date = None
                link_news = None
            
            article_page = requests.get(url=link_news) # news page
            while True:
                if article_page.status_code == 200:
                    article_page = article_page.text
                    break
                else:
                    print(current_page.status_code)
                
            soup2 = bs(article_page, 'html.parser')
            sleep(5)
            
            try:
                content = soup2.find('div', {'itemprop' : 'articleBody'}).text.replace('\n', '') # content
                category = soup2.find('li', {'class' : 'cate03'}).text # category
            except AttributeError as e:
                content = None
                category = None

            if title != None and headline != None and date != None and link_news != None and content != None and category != None:
                data.append([title, headline, date, link_news, content, category])
            
            print(len(data))
        
        idx_p += 1
        page_idx = '&pageIdx=' + str(idx_p)
        url_target = url_main + page_idx
            
    url_main = url + next_url

print()
print('----- End -----')

----- Start -----

page: 1


pageIdx=1

1
2
3
4
5
6
7
8
9
10

pageIdx=2

11
12
13
14
15
16
17
18
19
20

pageIdx=3

21
22
23
24
25
26
27
28
29
30

pageIdx=4

31
32
33
34
35
36
37
38
39
40

pageIdx=5

41
42
43
44
45
46
47
48
49
50

pageIdx=6

51
52
53
54
55
56
57
58
59
60

pageIdx=7

61
62
63
64
65
66
67
68


In [8]:
# res = requests.get(url=url_start).text
# soup = bs(res, 'html.parser')
# link = soup.find('link', {'rel' : 'canonical'})['href']
# date = link.split('?pageDate=')[-1]
# date

'20170101'

In [ ]:
dataset = pd.DataFrame(data=data, columns=columns)
dataset.to_csv('./kbs.csv', index=False)

In [ ]:
# span class read : 헤드라인 ~
# span class thumb : 썸네일 ~
# strong class sub : 뉴스 제목 ~
# span class data : 뉴스 날짜 및 시간 ~ 
# meta itemid : 뉴스 링크 ~


# li class cate03 : 카테고리 ~
# div class itemprop articleBody : 뉴스 내용

# div class mdp_inner -> a href #page : 
# div class w_news_list type_issue : 뉴스 정보

In [9]:
current_page = requests.get(url = url_main)
print(current_page.status_code)
# soup = bs(current_page, 'html.parser')


# list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'}))

# link_news = list_news[0].find('img')['src'] 



200
